In [1]:
import os, sys, glob
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from pathlib import Path
from src.utils import get_proj_root
import pandas as pd
import numpy as np
import detect_modality, find_thresholds, create_synthetic_data
import explore_anonymize_caida
import warnings

warnings.filterwarnings("ignore")
random_seed = 222


    Anonymize datasets ...     


In [2]:
proj_root = get_proj_root()
path_proj = proj_root 
path_proj_pu = proj_root / 'project'
# path_caida = proj_root / 'pu_caida' / 'caida_datasets'
input_dir = path_proj / 'caida_processed'
# thres_dir = path_proj_pu / 'caida_processed'
current_dir = path_proj 

#Creating a folder if it does not exist
output_dir = path_proj_pu / 'caida_processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
# go to the original caida datasets
os.chdir(input_dir)
# collect the caida_data in the csv format
caida_data = glob.glob('*.csv')
print("caida data: ", caida_data)
# sort the dataset name to make it easier to track
caida_sorted = sorted(caida_data)
print("caida sorted: ", caida_sorted)
# example use only the first dataset
datasets = caida_sorted[0]
print("datasets: ", datasets)
anon_caida_name = []

caida data:  ['dataset_023.csv', 'dataset_018.csv', 'dataset_011.csv', 'dataset_016.csv', 'dataset_007.csv', 'dataset_000.csv', 'dataset_009.csv', 'dataset_017.csv', 'dataset_010.csv', 'dataset_019.csv', 'dataset_022.csv', 'dataset_008.csv', 'dataset_001.csv', 'dataset_006.csv', 'dataset_005.csv', 'dataset_002.csv', 'dataset_013.csv', 'dataset_014.csv', 'dataset_021.csv', 'dataset_003.csv', 'dataset_004.csv', 'dataset_020.csv', 'dataset_015.csv', 'dataset_012.csv']
caida sorted:  ['dataset_000.csv', 'dataset_001.csv', 'dataset_002.csv', 'dataset_003.csv', 'dataset_004.csv', 'dataset_005.csv', 'dataset_006.csv', 'dataset_007.csv', 'dataset_008.csv', 'dataset_009.csv', 'dataset_010.csv', 'dataset_011.csv', 'dataset_012.csv', 'dataset_013.csv', 'dataset_014.csv', 'dataset_015.csv', 'dataset_016.csv', 'dataset_017.csv', 'dataset_018.csv', 'dataset_019.csv', 'dataset_020.csv', 'dataset_021.csv', 'dataset_022.csv', 'dataset_023.csv']
datasets:  dataset_000.csv


In [18]:
import data_loader_latency
from sklearn.model_selection import train_test_split
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters
from scipy import sparse

In [5]:
os.chdir(output_dir / 'dataset_000')

In [6]:
%ls

dataset_000-s1.csv  Threshold-s1-dataset_000.csv
dataset_000-s2.csv  Threshold-s2-dataset_000.csv


In [9]:
data = pd.read_csv('dataset_000-s1.csv', names=['time','rtt','id'])

In [10]:
data['time'] = data['time'].apply(lambda x: str(x))

In [11]:
import tsfresh

In [19]:
extraction_settings = MinimalFCParameters()

In [20]:
data_extracted_feats = extract_features(data,\
                                            column_id='id', column_sort='time',\
                                            default_fc_parameters=extraction_settings,\
                                            impute_function=impute)

Feature Extraction: 100%|██████████████████████████████| 160/160 [00:00<00:00, 282.17it/s]


In [21]:
X_list = data_extracted_feats.values.tolist()

In [22]:
X = np.array(X_list)

In [24]:
train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, \
            train_original, val_original, test_original = data_loader_latency.split_data_2520(X, data)

In [25]:
df_threshold = pd.read_csv('Threshold-s1-dataset_000.csv')
thres = df_threshold['Threshold'][0]

In [27]:
def label(x, thres):
    if x >= thres:
        y = -1
    else:
        y = 1
    return y


In [28]:
# label test data
test_ground = test_original['rtt'].apply(label, thres=thres)

# label val data
val_ground = val_original['rtt'].apply(label, thres=thres)


In [63]:
test_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    320 non-null    object 
 1   rtt     320 non-null    float64
 2   id      320 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 7.6+ KB


In [29]:
train_pu = train_original.append(val_original)

In [72]:
train_primitive_matrix.shape

(704, 10)

In [75]:
train_primitive_matrix

array([[88.939, 88.939, 88.939, ..., 88.939, 88.939, 88.939],
       [88.986, 88.986, 88.986, ..., 88.986, 88.986, 88.986],
       [88.971, 88.971, 88.971, ..., 88.971, 88.971, 88.971],
       ...,
       [88.905, 88.905, 88.905, ..., 88.905, 88.905, 88.905],
       [88.846, 88.846, 88.846, ..., 88.846, 88.846, 88.846],
       [88.986, 88.986, 88.986, ..., 88.986, 88.986, 88.986]])

In [73]:
val_primitive_matrix.shape

(256, 10)

In [78]:
train_prim_pu = np.concatenate([train_primitive_matrix, val_primitive_matrix], axis=0)

In [79]:
train_prim_pu

array([[88.939, 88.939, 88.939, ..., 88.939, 88.939, 88.939],
       [88.986, 88.986, 88.986, ..., 88.986, 88.986, 88.986],
       [88.971, 88.971, 88.971, ..., 88.971, 88.971, 88.971],
       ...,
       [89.174, 89.174, 89.174, ..., 89.174, 89.174, 89.174],
       [88.933, 88.933, 88.933, ..., 88.933, 88.933, 88.933],
       [89.236, 89.236, 89.236, ..., 89.236, 89.236, 89.236]])

In [81]:
train_prim_pu.shape

(960, 10)

In [30]:
len(train_original)

704

In [31]:
len(val_original)

256

In [33]:
test_ground.info()

<class 'pandas.core.series.Series'>
RangeIndex: 320 entries, 0 to 319
Series name: rtt
Non-Null Count  Dtype
--------------  -----
320 non-null    int64
dtypes: int64(1)
memory usage: 2.6 KB


In [44]:
y_train = np.full((len(train_original), ), -1., dtype=int)

In [45]:
y_train

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1

In [46]:
test_ground

0      1
1      1
2      1
3     -1
4      1
      ..
315    1
316    1
317    1
318    1
319    1
Name: rtt, Length: 320, dtype: int64

In [131]:
unique, counts = np.unique(test_ground, return_counts=True)

In [132]:
np.asarray((unique,counts)).T

array([[ -1,  16],
       [  1, 304]])

In [137]:
unique, counts = np.unique(val_ground, return_counts=True)

In [138]:
np.asarray((unique,counts)).T

array([[ -1,  22],
       [  1, 234]])

In [47]:
y_train_df = pd.Series(y_train)

In [49]:
y_train_new = y_train_df.append(val_ground)

In [82]:
y_train_new

0     -1
1     -1
2     -1
3     -1
4     -1
      ..
571   -1
572    1
573    1
574    1
575    1
Length: 960, dtype: int64

In [83]:
y_train_new.info()

<class 'pandas.core.series.Series'>
Int64Index: 960 entries, 0 to 575
Series name: None
Non-Null Count  Dtype
--------------  -----
960 non-null    int64
dtypes: int64(1)
memory usage: 15.0 KB


In [84]:
len(train_prim_pu)

960

In [85]:
len(y_train_new)

960

In [52]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

from pulearn import (
    ElkanotoPuClassifier,
    # WeightedElkanotoPuClassifier,
)

In [114]:
print("Training set contains {} examples.".format(len(y_train_new)))
print("{} are good data.".format(len(np.where(y_train_new == +1.)[0])))
print("{} are noise.".format(len(np.where(y_train_new == -1.)[0])))
pu_f1_scores = []
reg_f1_scores = []

Training set contains 960 examples.
234 are good data.
726 are noise.


In [115]:
n_sacrifice_iter = range(0, len(np.where(y_train_new == +1.)[0]))

In [116]:
print(n_sacrifice_iter)
print(len(n_sacrifice_iter))

range(0, 234)
234


In [117]:
n_sacrifice = len(np.where(y_train_new == +1.)[0])-1

In [118]:
n_sacrifice

233

In [119]:
train_prim_pu.shape[0]

960

In [120]:
len(train_prim_pu)

960

In [121]:
y_train_new

0     -1
1     -1
2     -1
3     -1
4     -1
      ..
571   -1
572    1
573    1
574    1
575    1
Length: 960, dtype: int64

In [122]:
y_train_new.shape

(960,)

In [126]:
unique, counts = np.unique(y_train_new, return_counts=True)

In [127]:
np.asarray((unique, counts)).T

array([[ -1, 726],
       [  1, 234]])

In [144]:
for n_sacrifice in n_sacrifice_iter:
    print("PU transformation in progress...")
    print("Making {} good data examples noise.".format(n_sacrifice))
    y_train_pu = np.copy(y_train_new)
    pos = np.where(y_train_new == -1.)[0]
    np.random.shuffle(pos)
    sacrifice = pos[:n_sacrifice+1]
    y_train_pu[sacrifice] = +1.
    pos = len(np.where(y_train_pu == +1.)[0])
    unlabelled = len(np.where(y_train_pu == -1.)[0])
    print("PU transformation applied. We now have:")
    print("{} are noise.".format(len(np.where(y_train_pu == -1.)[0])))
    print("{} are good data.".format(len(np.where(y_train_pu == +1.)[0])))
    print("-------------------")
    print((
        "Fitting PU classifier (using a random forest as an inner "
        "classifier)..."
    ))
    estimator = RandomForestClassifier(
        n_estimators=100,
        criterion='gini',
        bootstrap=True,
        n_jobs=1,
    )
    # pu_estimator = WeightedElkanotoPuClassifier(
    #    estimator, pos, unlabelled)
    pu_estimator = ElkanotoPuClassifier(estimator)
#     print(pu_estimator)
#     print("train_prim_pu shape: ", train_prim_pu.shape)
#     print("y_train_pu shape: ", y_train_pu.shape)
    try:
        pu_estimator.fit(train_prim_pu, y_train_pu)
        y_pred = pu_estimator.predict(test_primitive_matrix)
        precision, recall, f1_score, _ = precision_recall_fscore_support(
            test_ground, y_pred)
        pu_f1_scores.append(f1_score[1])
        print("F1 score: {}".format(f1_score[1]))
        print("Precision: {}".format(precision[1]))
        print("Recall: {}".format(recall[1]))
#         print("Regular learning (w/ a random forest) in progress...")
#         estimator = RandomForestClassifier(
#             n_estimators=100,
#             bootstrap=True,
#             n_jobs=1,
#         )
#         estimator.fit(train_prim_pu, y_train_pu)
#         y_pred = estimator.predict(test_primitive_matrix)
#         precision, recall, f1_score, _ = precision_recall_fscore_support(
#             test_ground, y_pred)
#         reg_f1_scores.append(f1_score[1])
#         print("F1 score: {}".format(f1_score[1]))
#         print("Precision: {}".format(precision[1]))
#         print("Recall: {}".format(recall[1]))
    except ValueError:
        break

PU transformation in progress...
Making 0 good data examples noise.
PU transformation applied. We now have:
725 are noise.
235 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6695842450765863
Precision: 1.0
Recall: 0.5032894736842105
PU transformation in progress...
Making 1 good data examples noise.
PU transformation applied. We now have:
724 are noise.
236 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.72
Precision: 1.0
Recall: 0.5625
PU transformation in progress...
Making 2 good data examples noise.
PU transformation applied. We now have:
723 are noise.
237 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6839826839826839
Precision: 1.0
Recall: 0.5197368421052632
PU transformation in progress...
Making 3 good data examples noise.
PU transformation applied. We now have:
722

F1 score: 0.702127659574468
Precision: 0.9939759036144579
Recall: 0.5427631578947368
PU transformation in progress...
Making 28 good data examples noise.
PU transformation applied. We now have:
697 are noise.
263 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.711864406779661
Precision: 1.0
Recall: 0.5526315789473685
PU transformation in progress...
Making 29 good data examples noise.
PU transformation applied. We now have:
696 are noise.
264 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6908315565031984
Precision: 0.9818181818181818
Recall: 0.5328947368421053
PU transformation in progress...
Making 30 good data examples noise.
PU transformation applied. We now have:
695 are noise.
265 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.72
Precision: 1.0
Recall: 0.5625
PU transf

F1 score: 0.7412008281573499
Precision: 1.0
Recall: 0.5888157894736842
PU transformation in progress...
Making 56 good data examples noise.
PU transformation applied. We now have:
669 are noise.
291 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7530364372469636
Precision: 0.9789473684210527
Recall: 0.6118421052631579
PU transformation in progress...
Making 57 good data examples noise.
PU transformation applied. We now have:
668 are noise.
292 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.746938775510204
Precision: 0.9838709677419355
Recall: 0.6019736842105263
PU transformation in progress...
Making 58 good data examples noise.
PU transformation applied. We now have:
667 are noise.
293 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7329192546583851
Precision: 0.98882681564

F1 score: 0.7331975560081467
Precision: 0.9625668449197861
Recall: 0.5921052631578947
PU transformation in progress...
Making 82 good data examples noise.
PU transformation applied. We now have:
643 are noise.
317 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7586206896551725
Precision: 0.9894179894179894
Recall: 0.6151315789473685
PU transformation in progress...
Making 83 good data examples noise.
PU transformation applied. We now have:
642 are noise.
318 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7530364372469636
Precision: 0.9789473684210527
Recall: 0.6118421052631579
PU transformation in progress...
Making 84 good data examples noise.
PU transformation applied. We now have:
641 are noise.
319 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8084291187739464
Precisio

F1 score: 0.7914230019493177
Precision: 0.9712918660287081
Recall: 0.6677631578947368
PU transformation in progress...
Making 109 good data examples noise.
PU transformation applied. We now have:
616 are noise.
344 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7469879518072289
Precision: 0.9587628865979382
Recall: 0.6118421052631579
PU transformation in progress...
Making 110 good data examples noise.
PU transformation applied. We now have:
615 are noise.
345 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.726530612244898
Precision: 0.956989247311828
Recall: 0.5855263157894737
PU transformation in progress...
Making 111 good data examples noise.
PU transformation applied. We now have:
614 are noise.
346 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.756
Precision: 0.9642857

F1 score: 0.813102119460501
Precision: 0.9813953488372092
Recall: 0.694078947368421
PU transformation in progress...
Making 135 good data examples noise.
PU transformation applied. We now have:
590 are noise.
370 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.768
Precision: 0.9795918367346939
Recall: 0.631578947368421
PU transformation in progress...
Making 136 good data examples noise.
PU transformation applied. We now have:
589 are noise.
371 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7624750499001995
Precision: 0.9695431472081218
Recall: 0.6282894736842105
PU transformation in progress...
Making 137 good data examples noise.
PU transformation applied. We now have:
588 are noise.
372 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7595959595959596
Precision: 0.98429319

F1 score: 0.7882352941176471
Precision: 0.9757281553398058
Recall: 0.6611842105263158
PU transformation in progress...
Making 162 good data examples noise.
PU transformation applied. We now have:
563 are noise.
397 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7768924302788845
Precision: 0.9848484848484849
Recall: 0.6414473684210527
PU transformation in progress...
Making 163 good data examples noise.
PU transformation applied. We now have:
562 are noise.
398 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7722772277227724
Precision: 0.9701492537313433
Recall: 0.6414473684210527
PU transformation in progress...
Making 164 good data examples noise.
PU transformation applied. We now have:
561 are noise.
399 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7821011673151751
Preci

F1 score: 0.8107074569789674
Precision: 0.9680365296803652
Recall: 0.6973684210526315
PU transformation in progress...
Making 189 good data examples noise.
PU transformation applied. We now have:
536 are noise.
424 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7923809523809523
Precision: 0.9411764705882353
Recall: 0.6842105263157895
PU transformation in progress...
Making 190 good data examples noise.
PU transformation applied. We now have:
535 are noise.
425 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8136882129277567
Precision: 0.963963963963964
Recall: 0.7039473684210527
PU transformation in progress...
Making 191 good data examples noise.
PU transformation applied. We now have:
534 are noise.
426 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7713717693836979
Precis

F1 score: 0.8167938931297711
Precision: 0.9727272727272728
Recall: 0.7039473684210527
PU transformation in progress...
Making 215 good data examples noise.
PU transformation applied. We now have:
510 are noise.
450 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8317580340264651
Precision: 0.9777777777777777
Recall: 0.7236842105263158
PU transformation in progress...
Making 216 good data examples noise.
PU transformation applied. We now have:
509 are noise.
451 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8054474708171206
Precision: 0.9857142857142858
Recall: 0.680921052631579
PU transformation in progress...
Making 217 good data examples noise.
PU transformation applied. We now have:
508 are noise.
452 are good data.
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8250950570342206
Precis

In [162]:
os.chdir(path_proj)
import pu_learn_fit as plf
import data_loader_latency as dl

In [164]:
os.chdir(output_dir / 'dataset_000')

In [150]:
data = pd.read_csv('dataset_000-s1.csv', names=['time','rtt','id'])
data['time'] = data['time'].apply(lambda x: str(x))
extraction_settings = MinimalFCParameters()
data_extracted_feats = extract_features(data,\
                                            column_id='id', column_sort='time',\
                                            default_fc_parameters=extraction_settings,\
                                            impute_function=impute)
X_list = data_extracted_feats.values.tolist()
X = np.array(X_list)

Feature Extraction: 100%|██████████████████████████████| 160/160 [00:00<00:00, 434.44it/s]


In [152]:
train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, \
            train_original, val_original, test_original = dl.split_data_2520(X, data)

In [153]:
df_threshold = pd.read_csv('Threshold-s1-dataset_000.csv')
thres = df_threshold['Threshold'][0]

In [154]:
def label(x, thres):
    if x >= thres:
        y = -1
    else:
        y = 1
    return y

In [155]:
# label test data
test_ground = test_original['rtt'].apply(label, thres=thres)

# label val data
val_ground = val_original['rtt'].apply(label, thres=thres)

In [157]:
X_train_new = plf.concatenate_train_val(train_primitive_matrix, val_primitive_matrix)

In [167]:
y_train_init = np.full(len(train_original,), -1., dtype=int)

In [168]:
y_train_new = plf.create_new_y_train(y_train_init, val_ground)

In [171]:
plf.calcute_pu_scores(y_train_new, X_train_new, test_primitive_matrix, test_ground)

PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6980728051391862
Precision: 1.0
Recall: 0.5361842105263158
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6518847006651884
Precision: 1.0
Recall: 0.48355263157894735
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7063829787234043
Precision: 1.0
Recall: 0.5460526315789473
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6428571428571429
Precision: 1.0
Recall: 0.47368421052631576
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7008547008547009
Precision: 1.0
Recall: 0.5394736842105263
PU transformation 

F1 score: 0.7226890756302521
Precision: 1.0
Recall: 0.5657894736842105
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6909871244635193
Precision: 0.9938271604938271
Recall: 0.5296052631578947
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.6965811965811967
Precision: 0.9939024390243902
Recall: 0.5361842105263158
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7525562372188139
Precision: 0.9945945945945946
Recall: 0.6052631578947368
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7751004016064257
Precision: 0.9948453608247423
Recall: 0.6348684210526315
PU transformation in progress...
-------------------
Fitting PU classifier (using a r

F1 score: 0.7227926078028748
Precision: 0.9617486338797814
Recall: 0.5789473684210527
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7689243027888445
Precision: 0.9747474747474747
Recall: 0.6348684210526315
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7048832271762207
Precision: 0.9940119760479041
Recall: 0.5460526315789473
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7342799188640975
Precision: 0.9576719576719577
Recall: 0.5953947368421053
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7740667976424361
Precision: 0.9609756097560975
Recall: 0.6480263157894737
PU transformation in progress...
-------------------
Fitting PU classi

F1 score: 0.7459016393442623
Precision: 0.9891304347826086
Recall: 0.5986842105263158
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8132295719844358
Precision: 0.9952380952380953
Recall: 0.6875
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8122605363984675
Precision: 0.9724770642201835
Recall: 0.6973684210526315
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7540322580645162
Precision: 0.9739583333333334
Recall: 0.6151315789473685
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7357723577235772
Precision: 0.9627659574468085
Recall: 0.5953947368421053
PU transformation in progress...
-------------------
Fitting PU classifier (using 

F1 score: 0.7866927592954991
Precision: 0.9710144927536232
Recall: 0.6611842105263158
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8261682242990653
Precision: 0.9567099567099567
Recall: 0.7269736842105263
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.787524366471735
Precision: 0.9665071770334929
Recall: 0.6644736842105263
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.79296875
Precision: 0.9759615384615384
Recall: 0.6677631578947368
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7844660194174756
Precision: 0.957345971563981
Recall: 0.6644736842105263
PU transformation in progress...
-------------------
Fitting PU classifier (usin

F1 score: 0.7999999999999999
Precision: 0.976303317535545
Recall: 0.6776315789473685
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.7731755424063117
Precision: 0.9655172413793104
Recall: 0.6447368421052632
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8093385214007782
Precision: 0.9904761904761905
Recall: 0.6842105263157895
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.8160919540229885
Precision: 0.9770642201834863
Recall: 0.7006578947368421
PU transformation in progress...
-------------------
Fitting PU classifier (using a random forest as an inner classifier)...
F1 score: 0.782608695652174
Precision: 0.9801980198019802
Recall: 0.6513157894736842
PU transformation in progress...
-------------------
Fitting PU classifi